In [17]:
import torch
def area(x):
    y = torch.prod(torch.roll(x[:,1,:],shifts=-1,dims=1), dim = 1)
    z = torch.prod(torch.roll(x[:,1,:],shifts=1,dims=1), dim = 1)

    return (torch.abs(y-z)/2.)[None]
area(torch.ones(30,2,10)).shape

torch.Size([1, 30])

In [34]:
import torch

def hausdorff_distance(polygons1, polygons2):
    """
    Computes the Hausdorff distance between two batches of 2D polygons.

    Parameters:
    polygons1 (torch.Tensor): A tensor of shape (B, 2, N) representing the first batch of polygons,
                        where B is the batch size and N is the number of points in each polygon.
    polygons2 (torch.Tensor): A tensor of shape (B, 2, N) representing the second batch of polygons,
                        where B is the batch size and N is the number of points in each polygon.

    Returns:
    torch.Tensor: A 1D tensor of shape (B,) containing the Hausdorff distance for each pair of polygons.
    """
    B, _, N = polygons1.shape

    # Compute pairwise distances
    polygons1 = polygons1.permute(0, 2, 1)  # (B, N, 1, 2)
    polygons2 = polygons2.permute(0, 2, 1)  # (B, 1, N, 2)

    dists = torch.cdist(polygons1, polygons2)  # (B, N, N)
    # Compute the directed Hausdorff distances
    min_dist_polygons1_to_polygons2, _ = torch.min(dists, dim=2)  # (B, N)
    min_dist_polygons2_to_polygons1, _ = torch.min(dists, dim=1)  # (B, N)

    # Max of the minimum distances
    hausdorff_dist_polygons1_to_polygons2 = torch.max(min_dist_polygons1_to_polygons2, dim=1).values  # (B,)
    hausdorff_dist_polygons2_to_polygons1 = torch.max(min_dist_polygons2_to_polygons1, dim=1).values  # (B,)

    # Final Hausdorff distance is the maximum of both directed distances
    hausdorff_dist = torch.max(hausdorff_dist_polygons1_to_polygons2, hausdorff_dist_polygons2_to_polygons1)

    return hausdorff_dist

batch1 = torch.tensor([
    [[0, 0], [1, 0], [1, 1], [0, 1]],  # Square
    [[0, 0], [2, 0], [2, 1], [0, 1]]   # Rectangle
], dtype=torch.float32).permute(0, 2, 1)  # Permute to shape (B, 2, N)
batch2 = torch.tensor([
    [[0, 0], [1, 0], [1, 1], [0, 1]],  # Another Square
    [[0, 0], [2, 0], [2, 2], [0, 2]]   # Another Rectangle
], dtype=torch.float32).permute(0, 2, 1)  # Permute to shape (B, 2, N)
# print(batch2.shape)

# Calculate the Hausdorff distance between the batches of polygons
hausdorff_dists = hausdorff_distance(batch1, batch2)

print(hausdorff_dists.shape) 

torch.Size([2, 2, 4])
torch.Size([2, 4, 4])
torch.Size([2])
